In [1]:
import os
import pandas as pd
from glob2 import glob, iglob
from tqdm import tqdm
from music21.midi.translate import midiFilePathToStream
from multiprocessing import Pool
from timeout import timeout
import hashlib
import inspect
import pdb
import mido

 
Music21 v.4 is the last version that will support Python 2.
Please start using Python 3 instead.

Set music21.environment.UserSettings()['warnings'] = 0
to disable this message.



## Inside Label File
 - Number of instruments
 - Instrument names
 - Original file name
 - Length in seconds
 - Number of midi events

In [2]:
batch = glob('v1/*.[mM][iI][dD]')
print('There are', len(batch), 'in folder.')

('There are', 194416, 'in folder.')


In [3]:
out_midi_dir = 'v2/midis/'
out_label_dir = 'v2/labels/'

In [4]:
errors = []

def wrapper(tup):
    path, id_ = tup
    try:
        process(path, id_)
    except Exception as e:
        errors.append(e)

@timeout(50)
def process(path, id_):
    ##### CREATE NAMES
    f_name = os.path.split(path)[-1]
    hash_ = hashlib.md5(open(path, 'rb').read()).hexdigest()
    label_out = out_label_dir+hash_+'.csv'
    midi_out = out_midi_dir+hash_+'.mid'
    if os.path.exists(midi_out) and os.path.exists(label_out):
        # Return if already created
        return
    #### READ IN MIDIS AND GET INFO
    mid = mido.MidiFile(path)
    instruments = [x.name for x in mid.tracks]
    num_tracks = len(instruments)
    instruments = filter(lambda x: x != '', instruments)
    label = pd.DataFrame({'instruments': [','.join([x.replace(',', '\,') for x in instruments])],
                          'length': [mid.length],
                          'num_tracks': [num_tracks],
                          'num_nonempty_tracks': [len(instruments)],
                          'original_fname': [f_name],
                          'num_midi_events': [max([len(x) for x in mid.tracks])],
                          })
    label.to_csv(label_out, index=False)
    mid.save(midi_out)

In [ ]:
# map(wrapper, tqdm(zip(batch, range(len(batch)))))
p = Pool(30)
for i, _ in tqdm(enumerate(p.imap_unordered(wrapper, zip(batch, range(len(batch))))), total=len(batch)):
    pass
pool.close()

  4%|▍         | 7785/194416 [11:58<4:46:57, 10.84it/s]